In [1]:
#!pip install webdriver_manager
#!pip install lxml
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re
from time import sleep
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager



## PART 1: Write a function that gets the URLs of Money Diary stories

In [2]:
#Testing

r = requests.get("https://www.refinery29.com/en-us/money-diary?page=35")
parsed = bs(r.text, 'html.parser')
#This uses find_all to get everything that is <a> class (which in html references the hyperlinks)
#Then it filters the 
#results=set(filter(lambda x: re.search("R29 ?Team", x.text) is not None, parsed.find_all('a')))
#results=set(filter(lambda x: re.search("en-us", x.get('href')), parsed.find_all('a')))
#results=set(filter(lambda x: re.search("en-us.*money-diary[^?]", x.get('href')), parsed.find_all('a')))
#results=set(filter(lambda x: re.search("en-us.*money-diary[^extras]", x.get('href')), parsed.find_all('a')))

results=set(filter(lambda x: "extras" not in x.get('href') and "money-diar" in x.get('href') and "?" not in x.get('href') and "signup" not in x.get('href'), parsed.find_all('a')))

#print(results)
for item in results:
    print("-----")
    #Headline text
    print(item.text)
    #URL 
    print(item.get('href'))


-----
Money DiariesA Week In Washington, D.C., On A $45,000 SalaryWelcome to Money Diaries, where we're tackling what might be the last taboo facing modern working women: money. We're asking millennials how theyby You
/en-us/money-diary-washington-dc-assistant-to-vp-salary
-----
Money DiariesA Week In Shanghai, China, On A $36,000 SalaryWelcome to Money Diaries, where we're tackling what might be the last taboo facing modern working women: money. We're asking millennials how theyby You
/en-us/money-diary-shanghai-china-copywriter-salary
-----
Money DiariesA Week In Lincoln, NE, On A $21,000 Salary" I already have lentils at home, so I get brussels sprouts, tomatoes, bananas, avocados, mangoes, cantaloupe, olive oil, balsamic vinegar, bread, tortillaby You
/en-us/money-diary-lincoln-ne-graduate-research-assistant-salary
-----
Money DiariesWhy I Moved From The City To A Town Of 600by Anabel Pasarow
/en-us/money-diaries-interview-country-living
-----
Enter to WinMoney Diary Lover? We’re G

In [3]:
#This initializes the set of links that I will eventually populate
diary_links = set()

#A.  Defines a function that, given a url (to a Money Diaries main page), gets the links of the associated stories on 
#that page.  Input is the variable url
def get_diary_links_from_url(url):
    """
    Fetches all diary links from a given url, otherwise returns None.
    """
    #goes to the url and stores the stuff on that url as the variable r
    r = requests.get(url)
    # refinery29 returns a 403 for pages that don't exist
    if r.status_code == 403:
        return None
    #Run beautiful soup -- the input is r.text (which textifies the request from the url)
    #bs(input,'html.parser' ) -- 'html.parser' is the BS thing that parses the website
    parsed = bs(r.text, 'html.parser')
    #This uses find_all to get everything that is <a> class (which in html references the hyperlinks)
    #return set(filter(lambda x: re.search("R29 ?Team", x.text), parsed.find_all('a')))
    #there is a broken link with extras in it.
    #return set(filter(lambda x: re.search("en-us.*money-diar", x.get('href')) is not None, parsed.find_all('a')))
    return set(filter(lambda x: "money-diar" in x.get('href') and "extras" not in x.get('href') and "submit" not in x.get('href') and "?" not in x.get('href') and "signup" not in x.get('href'), parsed.find_all('a')))



In [4]:

#B.  This initializes the actual running of the function we just defined
page_number = 1
MONEY_DIARY_ROOT = "https://www.refinery29.com/en-us/money-diary"
links = True
while links is not None:
    print(f"Getting links from page {page_number}")
    links = get_diary_links_from_url(f"{MONEY_DIARY_ROOT}?page={page_number}")
    if links:
        links = {l.get('href') for l in links}
        print(f"Got {len(links)} from page {page_number} ({len(links.difference(diary_links))} new)")
        diary_links.update(links)
    page_number = page_number + 1
    sleep(0.1)


Getting links from page 1
Got 34 from page 1 (34 new)
Getting links from page 2
Got 30 from page 2 (26 new)
Getting links from page 3
Got 29 from page 3 (29 new)
Getting links from page 4
Got 29 from page 4 (29 new)
Getting links from page 5
Got 30 from page 5 (30 new)
Getting links from page 6
Got 30 from page 6 (30 new)
Getting links from page 7
Got 29 from page 7 (29 new)
Getting links from page 8
Got 30 from page 8 (30 new)
Getting links from page 9
Got 30 from page 9 (30 new)
Getting links from page 10
Got 29 from page 10 (29 new)
Getting links from page 11
Got 30 from page 11 (30 new)
Getting links from page 12
Got 29 from page 12 (29 new)
Getting links from page 13
Got 29 from page 13 (29 new)
Getting links from page 14
Got 30 from page 14 (30 new)
Getting links from page 15
Got 30 from page 15 (30 new)
Getting links from page 16
Got 29 from page 16 (29 new)
Getting links from page 17
Got 29 from page 17 (29 new)
Getting links from page 18
Got 29 from page 18 (29 new)
Getting li

In [5]:
diary_link_list = list(diary_links)
print(len(diary_link_list))
#print(diary_link_list[2])
diary_links

1495


{'/en-us/teacher-tri-cities-wa-salary-money-diary',
 '/en-us/money-diary-fashion-merchandise-budget',
 '/en-us/money-diaries-couple-memphis-interview',
 '/en-us/money-diary-lincoln-nebraska-public-relations-intern-salary',
 '/en-us/human-resources-northern-indiana-salary-money-diary',
 '/en-us/school-leadership-tennessee-salary-money-diary',
 '/en-us/money-diary-events-coordinator-paris-budget',
 '/en-us/architect-philadelphia-pa-salary-money-diary',
 '/en-us/money-diary-atlanta-ga-phd-candidate-in-genetics-grad-student-salary',
 '/en-us/corrections-supervisor-madison-wi-salary-money-diary',
 '/en-us/pastry-chef-los-angeles-ca-salary-money-diary',
 '/en-us/money-diary-holland-mi-administrative-assistant-salary',
 '/en-us/money-diary-melbourne-australia-doctor-salary',
 '/en-us/investment-banking-analyst-new-york-ny-salary-money-diary',
 '/en-us/bahrain-ceo-producer-salary-money-diary',
 '/en-us/project-manager-detroit-mi-salary-money-diary',
 '/en-us/money-diary-london-birmingham-hr-ad

## PART 2A: Write function that gets the content of the links

In [6]:
#This creates a function to get the content of links
def get_editorial_content(diary_link):
    print("Getting content from url", diary_link)
    r = requests.get(diary_link)
    if r.status_code != 200:
        print(f"Invalid link: {diary_link}")
        return None
    parsed = bs(r.text, 'html.parser')
    #This is the 
    content = parsed.find_all(class_='section-text')
    if content is None:
        print(f"No .section-text(s) at {diary_link}")
    return [section.text for section in content if section.text is not None]

#This tests it out to make sure it worked
get_editorial_content(f"https://www.refinery29.com{diary_link_list[2]}")

Getting content from url https://www.refinery29.com/en-us/money-diaries-couple-memphis-interview


["Yesterday, we published the Money Diary of a couple in Memphis, TN, who makes a combined $133,000 per year. A. is a communications director, and her husband, C., is a public defender. They live in a house they own in Memphis, and spent the week of their Money Diary celebrating A.'s 34th birthday.",
 "In celebration of Money Diaries Month, we talked to A. and C. about what they thought of each other's Money Diaries, their spending philosophies, and what's next for them. ",
 'Want even more Money Diaries, plus $$$ advice from a kick-ass, all-female team of financial advisors, and tips on how to save more than $500? Pick up our new book: Money Diaries! Order here!',
 'What did you think of each other’s Money Diaries? Were you surprised by anything?',
 "A: Mostly when I read C.'s I was like, man, I should have been funnier in mine.",
 'C: I wasn’t surprised by anything. I guess after being together for so long you kind of know and accept what the other person is spending money on.',
 'A:

## PART 2B: Get the number of comments

In [19]:
#This is done w/ selenium so we need to initialize the driver
#driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======


2022-07-09 00:31:36,424 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 103.0.5060


2022-07-09 00:31:36,484 INFO Current google-chrome version is 103.0.5060


[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome


...

[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_mac64_m1.zip


2022-07-09 00:31:36,547 INFO About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_mac64_m1.zip


[WDM] - Driver has been saved in cache [/Users/danielleli/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.53]


2022-07-09 00:31:36,884 INFO Driver has been saved in cache [/Users/danielleli/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.53]


/var/folders/fy/56t7kzwn4m196p75kksy9j5c0000gn/T/ipykernel_35439/3905140457.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [8]:
#Just check to make sure I can get the comments on a single link

# driver.get("https://www.refinery29.com/en-us/paralegal-brooklyn-ny-salary-money-diary")
# time.sleep(5)
# driver.execute_script("""
#     window.scrollTo({ top: document.body.scrollHeight, behavior: 'smooth' })
# """)
# time.sleep(1)
# comment_count = driver.execute_script("""
#     return document.querySelector("[data-spot-im-shadow-host*='conversation']").shadowRoot.querySelector("[data-spot-im-class='comments-count']").textContent
# """)

# comment_count


'181 Comments'

In [27]:
#This defines the function
def get_num_comments(url):
    """
    Fetches number of comments for a given url
    """
    print("Getting comments from url", url)
    driver.get(url)

    time.sleep(15)
    print("Scrolling down")
    driver.execute_script("""
    window.scrollTo({ top: document.body.scrollHeight, behavior: 'smooth' })
    """)
    time.sleep(15)

    print("Getting comment count")
    comment_count = driver.execute_script("""
    try {
        return document.querySelector("[data-spot-im-shadow-host*='conversation']").shadowRoot.querySelector("[data-spot-im-class='comments-count']").textContent
    } catch(e) {
        return null
    }
    """)
    
    return comment_count

## PART 3: Loop through all of the links to get the content and comments

In [28]:
#This is done w/ selenium so we need to initialize the driver
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======


2022-07-09 08:17:46,367 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 103.0.5060


2022-07-09 08:17:46,436 INFO Current google-chrome version is 103.0.5060


[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome


2022-07-09 08:17:46,437 INFO Get LATEST chromedriver version for 103.0.5060 google-chrome


[WDM] - Driver [/Users/danielleli/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.53/chromedriver] found in cache


2022-07-09 08:17:46,498 INFO Driver [/Users/danielleli/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.53/chromedriver] found in cache


/var/folders/fy/56t7kzwn4m196p75kksy9j5c0000gn/T/ipykernel_35439/3905140457.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [21]:
diary_link_list
len(diary_link_list)

1495

In [29]:
diaries=[]

#Loops over all the links to get the content 
for key, diary_link in enumerate(diary_link_list, 1):
    #GET THE CONTENT AND URL FIRST
    print(f"***{key}***")
    content = get_editorial_content(f"https://www.refinery29.com{diary_link}")
    url="https://www.refinery29.com"+diary_link
    
    #GET THE NUMBER OF COMMENTS NOW
    num_comments=get_num_comments(f"https://www.refinery29.com{diary_link}")
    
    diary_dict = {
        'url': url,
        'num_comments': num_comments,
        'content': content
    }
    diaries.append(diary_dict)
    sleep(2)



***1***
Getting content from url https://www.refinery29.com/en-us/teacher-tri-cities-wa-salary-money-diary
Getting comments from url https://www.refinery29.com/en-us/teacher-tri-cities-wa-salary-money-diary
Scrolling down
Getting comment count
***2***
Getting content from url https://www.refinery29.com/en-us/money-diary-fashion-merchandise-budget
Getting comments from url https://www.refinery29.com/en-us/money-diary-fashion-merchandise-budget
Scrolling down
Getting comment count
***3***
Getting content from url https://www.refinery29.com/en-us/money-diaries-couple-memphis-interview
Getting comments from url https://www.refinery29.com/en-us/money-diaries-couple-memphis-interview
Scrolling down
Getting comment count
***4***
Getting content from url https://www.refinery29.com/en-us/money-diary-lincoln-nebraska-public-relations-intern-salary
Getting comments from url https://www.refinery29.com/en-us/money-diary-lincoln-nebraska-public-relations-intern-salary
Scrolling down
Getting comment 

Scrolling down
Getting comment count
***33***
Getting content from url https://www.refinery29.com/en-us/money-diary-southern-connecticut-registered-nurse-salary
Getting comments from url https://www.refinery29.com/en-us/money-diary-southern-connecticut-registered-nurse-salary
Scrolling down
Getting comment count
***34***
Getting content from url https://www.refinery29.com/en-us/2020/04/9653876/asset-manager-rocky-mountains-coronavirus-salary-money-diary
Getting comments from url https://www.refinery29.com/en-us/2020/04/9653876/asset-manager-rocky-mountains-coronavirus-salary-money-diary
Scrolling down
Getting comment count
***35***
Getting content from url https://www.refinery29.com/en-us/nanny-washington-dc-salary-money-diary
Getting comments from url https://www.refinery29.com/en-us/nanny-washington-dc-salary-money-diary
Scrolling down
Getting comment count
***36***
Getting content from url https://www.refinery29.com/en-us/money-diary-new-orleans-la-associate-clinical-research-coordi

WebDriverException: Message: unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x000000010080dd14 chromedriver + 3792148
1   chromedriver                        0x00000001007a4828 chromedriver + 3360808
2   chromedriver                        0x00000001004a50b8 chromedriver + 217272
3   chromedriver                        0x0000000100494124 chromedriver + 147748
4   chromedriver                        0x00000001004938f4 chromedriver + 145652
5   chromedriver                        0x0000000100492f50 chromedriver + 143184
6   chromedriver                        0x00000001004927f8 chromedriver + 141304
7   chromedriver                        0x00000001004ab69c chromedriver + 243356
8   chromedriver                        0x00000001004fdd74 chromedriver + 580980
9   chromedriver                        0x00000001004ca318 chromedriver + 369432
10  chromedriver                        0x00000001007e31e8 chrom

## PART 4: Put everything into a dataframe

In [23]:
df=pd.DataFrame(diaries)
df.to_csv('moneydiaries.csv', index=False)
df.shape

(25, 3)

In [26]:
df.head(10)

count               24
unique              24
top       141 Comments
freq                 1
Name: num_comments, dtype: object